In [4]:
import os, json
import sys
import pandas as pd
import requests
from io import StringIO
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pd.options.plotting.backend = 'plotly'

for p in sys.path:
    if '99 scripts' in p.lower() :
        sys.path.remove(p)

sys.path.append('..')

import src

MainProcess


In [ ]:
query = """
SELECT * From Population
WHERE Age == 'Total'
"""
year = 1850
query = """
SELECT Sex, Age, "{year}" From Population
WHERE Age != 'Total' AND
Sex != 'Total' AND
Sex != 'Non-binary/Other'
""".format(year=year)

api = src.StatisticsIcelandAPI()
conn = api.conn

df = pd.read_sql(query, con=conn)

frame = (
    df.melt(id_vars=['Sex','Age'],var_name='Year',value_name='Population')
    .assign(Age=lambda s: s.Age.str.replace(r'\syears?','',regex=True).replace('Under 1','0').astype(int),
            Population=lambda s: s.Population.replace('..','0').astype(int)
            )
)

# Pivot for plotting
df_male = frame[frame['Sex'] == 'Males'].copy()
df_female = frame[frame['Sex'] == 'Females'].copy()
df_male['Population'] *= -1  # Invert male values for pyramid shape

pyramid_chart = go.Figure()

pyramid_chart.add_trace(go.Bar(
    y=df_male['Age'],
    x=df_male['Population'],
    orientation='h',
    name='Male',
    marker_color='blue'
))

pyramid_chart.add_trace(go.Bar(
    y=df_female['Age'],
    x=df_female['Population'],
    orientation='h',
    name='Female',
    marker_color='red'
))

pyramid_chart.update_layout(
    title='Population Pyramid',
    xaxis=dict(title='Population'),
    yaxis=dict(title='Age', categoryorder='category ascending'),
    barmode='overlay',
    bargap=0.1,
    template='plotly_white',
    title_x=0.5,
    showlegend=False,
    uirevision='None',
)    

In [26]:
query = """
SELECT "2022", "2023", "2024", "2025", Sex, Age From Population
WHERE Age == 'Total' AND 
Sex == 'Non-binary/Other'
"""

non_binary_chart = (
        pd.read_sql_query(query, conn)
        .melt(id_vars=['Sex','Age'],var_name='Year',value_name='Population')
        # .query('Age == "Total" & Sex == \'Non-binary/Other\' & Year >= "2022"')
        .assign(Year=lambda s: s.Year.astype(int),
                Population=lambda s: s.Population.replace('..','0').astype(int)
                )
        .plot(x ='Year',y='Population',title = 'Non-binary/Other citizens in Iceland 1841-2025',kind = 'bar')
        .update_layout(title_x = 0.5, uirevision = 'None',showlegend = False)
        .update_layout(title_font=dict(size=9))
        .update_yaxes(title_text = '')
        .update_xaxes(title_text = '')
)  
non_binary_chart

In [ ]:
# init my api obj
api = src.StatisticsIcelandAPI()

# this obj provides a typable and readable interface to getting the endpoint urls 
economyEndPoint = src.constants.ENDPOINTS.Efnahagur

# define what endpoints I want to request
request_endpoints = [
    economyEndPoint.Prices_and_consumption.Consumer_price_index.CPI.Consumer_price_index_and_changes__base_1988_100, # this is just a url endpoint
    economyEndPoint.Employment_and_labour_productivity.Employment.Number_of_employed_persons__jobs_and_hours_worked_by_economic_activity_and_quarters__1991_to_2024, # as is this
    getattr(economyEndPoint.National_accounts.Gross_domestic_product,'National_accounts_-_quarterly').Quarterly_GDP_1995_to_2024, # the map to typable didn't work well for this one
    economyEndPoint.National_accounts.Financial_accounts.Money_and_credit.Weighted_average_interest_rates_of_commercial_banks_1960_to_2016,

]

# pass to api
api.add_endpoints(request_endpoints)

# and request
response = api.request()

# retrieve
CPI = response['Consumer_price_index_and_changes__base_1988_100']
Employment = response['Number_of_employed_persons__jobs_and_hours_worked_by_economic_activity_and_quarters__1991_to_2024']
GDP = response['Quarterly_GDP_1995_to_2024']
Interest = response['Weighted_average_interest_rates_of_commercial_banks_1960_to_2016']

In [ ]:
api = src.StatisticsIcelandAPI()

request_endpoints = [
        src.constants.ENDPOINTS.Ibuar.Population.Overview.Quarterly_data.Births__deaths_and_migration_by_sex_and_citizenship__NUTS3_regions_and_quarters_2011_to_2024,
        src.constants.ENDPOINTS.Ibuar.Population.Overview.Quarterly_data.Population_by_municipality__sex__citizenship_and_quarters_2011_to_2024,
        src.constants.ENDPOINTS.Ibuar.Population.Overview.Overview.Population_by_sex_and_age_1841_to_2025
        
    ]

api.add_endpoints(request_endpoints)

response = api.request()

Flux = response['Births__deaths_and_migration_by_sex_and_citizenship__NUTS3_regions_and_quarters_2011_to_2024']
Population_by_municipality = response['Population_by_municipality__sex__citizenship_and_quarters_2011_to_2024']
Population = response['Population_by_sex_and_age_1841_to_2025']


In [ ]:
df = convert_to_df(Flux)
(
    df.loc[lambda s: s.Event == 'Net migration',['Quarter','Total Total']]
    .plot(x='Quarter',y='Total Total',title='Net migration in Iceland 2011-2024')
)



In [ ]:
(
    df.loc[lambda s: (s.Event == 'Births')+(s.Event == 'Deaths'),['Quarter','Event','Total Total']]
    .plot(x='Quarter',y='Total Total',title='Births and Deaths in Iceland 2011-2024',color='Event')
)

In [ ]:


df = convert_to_df(Population_by_municipality)
(
    df.loc[lambda s: s.Municipality == 'Total',['Quarter','Municipality','Foreign citizens','Icelandic citizens']]
    .melt(id_vars=['Quarter','Municipality'])
    .assign(value = lambda s: s.value.astype(float))
    .plot(x = 'Quarter', y = 'value', color = 'variable')
)



In [ ]:
df = convert_to_df(Population)
(
    df.melt(id_vars=['Sex','Age'],var_name='Year',value_name='Population')
    .query('Age != "Total"')
    .assign(Age=lambda s: s.Age.str.replace(r'\syears?','',regex=True).replace('Under 1','0').astype(int),
            Year=lambda s: s.Year.astype(int),
            Population=lambda s: s.Population.replace('..','0').astype(int)
            )
    .groupby(['Year','Sex'])
    .agg({'Population':'sum'}).reset_index()
    .plot(x = 'Year',y='Population',color = 'Sex')
)

In [ ]:
frame = (
    df.melt(id_vars=['Sex','Age'],var_name='Year',value_name='Population')
    .query('Age != "Total" & Sex != \'Total\' & Sex != \'Non-binary/Other\'')
    .assign(Age=lambda s: s.Age.str.replace(r'\syears?','',regex=True).replace('Under 1','0').astype(int),
            Year=lambda s: s.Year.astype(int),
            Population=lambda s: s.Population.replace('..','0').astype(int)
            )
    .query('Year == 2024')    
)

# Pivot for plotting
df_male = frame[frame['Sex'] == 'Males'].copy()
df_female = frame[frame['Sex'] == 'Females'].copy()
df_male['Population'] *= -1  # Invert male values for pyramid shape

# Sort ages from youngest to oldest
age_categories = sorted(frame['Age'].unique())

fig = go.Figure()

fig.add_trace(go.Bar(
    y=df_male['Age'],
    x=df_male['Population'],
    orientation='h',
    name='Male',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    y=df_female['Age'],
    x=df_female['Population'],
    orientation='h',
    name='Female',
    marker_color='red'
))

fig.update_layout(
    title='Population Pyramid',
    xaxis=dict(title='Population'),
    yaxis=dict(title='Age', categoryorder='category ascending'),
    barmode='overlay',
    bargap=0.1,
    template='plotly_white'
)

fig.show()

In [ ]:
frame

In [ ]:
def convert_to_df(response:requests.Request)->pd.DataFrame:
    return pd.read_csv(
        StringIO(response.text.lstrip('ï»¿')),
        sep=',',
        encoding='utf-8'
    ) 

In [ ]:
def charts_Interest(Interest:requests.Request,variation:str)->go.Figure:
    return (
        convert_to_df(Interest)
        .replace(r'^\.+$','0.0', regex=True)
        .astype(float)
        .plot(x = 'Year', y = variation, title = variation, kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)
        .update_yaxes(title_text = '')
    )


charts_Interest(Interest,'General savings deposits Nominal interest, % per year')

In [ ]:

def charts_grossDomesticProduct(GDP:requests.Response,value_unit:str,Category)->go.Figure:
    return (
        convert_to_df(GDP)
        .set_index(['Value unit','Category'])
        .loc[pd.IndexSlice[value_unit, Category,:], :]#.T
        .reset_index().T
        .iloc[2:].reset_index(names = 'Quarter')
        .rename(columns = {0:Category})
        .assign(**{Category: lambda x: x[Category].astype(float)})
        .plot(x='Quarter', y=Category, title = f"{Category} in {value_unit}", kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)   
        .update_yaxes(title_text = '')
    )

charts_grossDomesticProduct(GDP,value_unit= 'Current prices', Category = '1. Private final consumption')

In [ ]:
def charts_consumerPriceIndex(CPI:requests.Response,variation:str)->go.Figure:
    return (
        convert_to_df(CPI)
        .plot(x = 'Month', y = variation, title = f"{variation} in Iceland", kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)
        .update_yaxes(title_text = '')
    )
charts_consumerPriceIndex(CPI, variation = 'Consumer price index Index')

In [ ]:

def charts_EmploymentBySector(Employment:requests.Response)->go.Figure:
    return (
        convert_to_df(Employment)
        .pipe(lambda s: s.set_index(s.columns[:3].tolist()))
        .loc[pd.IndexSlice['Jobs','Total employment'],]
        .drop('Total - All activities')
        .T
        .reset_index(names = 'time')
        .melt(id_vars=['time'], var_name ='sector', value_name='Number of Jobs')
        .plot(x = 'time', y = 'Number of Jobs', color = 'sector',kind = 'area')
        .update_layout(showlegend = False)
    )   

charts_EmploymentBySector(Employment)


In [ ]:
import re

def to_snake_case(name:str)->str:
    """Convert a string to snake_case."""
    name = re.sub(r'([a-z])([A-Z])', r'\1_\2', name)  # Add underscore between lowercase and uppercase letters
    name = re.sub(r'[^a-zA-Z0-9]+', '_', name)  # non-alphanumeric characters to underscore
    return name.lower().strip('_')  # Convert to lowercase

def create_SQL_command(resp:requests.Response, tableName:str)->str:
    mapping = {'object':'TEXT','int64':'INTEGER','float64':'REAL','string':'TEXT','Float64':'REAL','Int64':'INTEGER','boolean':'BOOLEAN'}
    df = convert_to_df(resp)
    # df.columns = [to_snake_case(col) for col in df.columns]
    df = df.replace(r'^\.+$','0', regex=True) # for some reason nan values arrive as . or .. 
    df = df.convert_dtypes()   
    dtypes = df.dtypes.replace(mapping).to_dict()

    sql_command = 'c.execute(\'DROP TABLE IF EXISTS {tableName};\')\n'
    sql_command += 'c.execute(\n\'\'\'\nCREATE TABLE IF NOT EXISTS {tableName} ('
    for column, dtype in dtypes.items():
        # snake_case = to_snake_case(column)
        sql_command += f"\n\t\"{column}\" {dtype},"
    sql_command = sql_command.rstrip(', \n') + '\n);\'\'\')\n\n'
    return sql_command.format(tableName = tableName)

CPI
Employment
GDP
Interest
# print(
#     create_SQL_command(CPI, 'CPI')
# )
# print(
#     create_SQL_command(Interest, 'Interest')
# )
# print(
#     create_SQL_command(Employment, 'Employment')
# )
# print(
#     create_SQL_command(GDP, 'GDP')
# )

print(
    create_SQL_command(Flux, 'Flux')
)
print(
    create_SQL_command(Population_by_municipality, 'Population_by_municipality')
)
print(
    create_SQL_command(Population, 'Population')
)



In [ ]:
convert_to_df(CPI).replace(r'^\.+$',None,regex=True).drop(columns = 'Month').astype(float)